# Data Cleaning 1

This is some basic inspection and cleaning for the twitter data. In this notebook we:
1. Find and mend two broken rows.
2. Find and drop duplicates.
3. Recast datatypes.
4. Add a datetime column for tweet creation timestamp.

### Import Dependencies

In [1]:
import pandas as pd

import pickle # just in case

### Set file locations

In [2]:
# Raw data file for tweet text (input file)
tweet_text_file = '../00_data/twitter_data/twitter_text.csv'

# File for cleaned tweet text (output file)
clean_csv = '../00_data/twitter_data/twitter_data_cleaned.csv'

### Read in raw data

In [3]:
df = pd.read_csv(tweet_text_file)

In [4]:
df.head()

,tweet_id,created_at,full_text,geo,coordinates,place,retweet_count,favorite_count,possibly_sensitive,lang,user_id
0,1364223054851813377,Tue Feb 23 14:38:16 +0000 2021,Here’s what's in the COVID relief package:\n \...,NaN,NaN,NaN,9160,38093,NOT FOUND,en,2.950125e+07
1,1364381497302671362,Wed Feb 24 01:07:52 +0000 2021,Will the National Endowment for the Arts be he...,NaN,NaN,NaN,6131,18560,NOT FOUND,en,1.201671e+18
2,1364609594056704002,Wed Feb 24 16:14:15 +0000 2021,"This is both anecdotal and early, but many lon...",NaN,NaN,NaN,5941,63174,NOT FOUND,en,3.842872e+07
3,1364726798412443649,Wed Feb 24 23:59:58 +0000 2021,A Link to Professor Chossudovsky’s Analysis of...,NaN,NaN,NaN,1,0,False,en,2.192010e+09
4,1364726797947052038,Wed Feb 24 23:59:58 +0000 2021,Children warned over hugging grandparents even...,NaN,NaN,NaN,0,2,False,en,2.868190e+09


In [5]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51172 entries, 0 to 51171
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            51172 non-null  object 
 1   created_at          51170 non-null  object 
 2   full_text           51170 non-null  object 
 3   geo                 77 non-null     object 
 4   coordinates         79 non-null     object 
 5   place               1008 non-null   object 
 6   retweet_count       51170 non-null  object 
 7   favorite_count      51170 non-null  object 
 8   possibly_sensitive  51170 non-null  object 
 9   lang                51168 non-null  object 
 10  user_id             51168 non-null  float64
dtypes: float64(1), object(10)
memory usage: 4.3+ MB


In [6]:
df.possibly_sensitive.value_counts()

False         30276
NOT FOUND     20438
True            454
46902592.0        1
46902592          1
Name: possibly_sensitive, dtype: int64

Observations:
1. 'geo', 'coordinates', and 'place' are very lightly populated. Exclude?
2. I would expect 'tweet_id', 'full_text', and 'user_id' to all have identical counts, but they do not. Need to look at that.
3. I would expect 'tweet_id' to be of the same Dtype as 'user_id', but it does not.
4. I would expect 'retweet_count' and 'favorite_count' to be integers, but they are not.
5. I would expect 'possibly_sensitive' to be free of numerical data.

### Check the obvious anomalies

We're looking to account for the holes in the counts above, i.e.:  
2 instances each of null values for 'created_at', 'full_text', 'retweet_count', 'favorite_count', and 'possibly_sensitive'  
4 instances each of null values for 'lang' and 'user_id'.  
2 instances of numerical data in 'possibly_sensitive'.

In [7]:
df[df['user_id'].isnull()]

,tweet_id,created_at,full_text,geo,coordinates,place,retweet_count,favorite_count,possibly_sensitive,lang,user_id
3600,1364723003620270083,Wed Feb 24 23:44:54 +0000 2021,Good News for Europeans... even if your leftis...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3601,https://t.co/dMoo7szXE0,NaN,NaN,NaN,1,1,False,en,46902592,NaN,NaN
16061,1366124610568216582,Sun Feb 28 20:34:23 +0000 2021,Important Open Letter and Analysis of the Covi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16062,https://t.co/AVf4W5aP3L,NaN,NaN,NaN,0,1,False,en,46902592.0,NaN,NaN


It looks like two broken lines in the csv. The 'full_text' entries somehow got split right before the included urls. This also explains why 'tweet_id', 'retweet_count', 'favorite_count', and 'user_id' had been automatically cast as strings, as well as why there is numerical data in 'possibly_sensitive'. That's everything we're looking for.

Looked these tweets up manually to see if there is anything unusual:  
twitter.com/anyuser/status/1364723003620270083  
twitter.com/anyuser/status/1366124610568216582  

Turns out it's the same user and sequence of hashtags for both tweets, but not the only time that user included that sequence of hashtags in our dataset. So, inconclusive (but repairable for now).

### Mend the broken rows

In [8]:
# We could also just drop them.
# df = df[df['user_id'].notna()]

In [9]:
# To mend this, we need to add the url back into the 'full_text' column and also sort of slide 
# the rest of the mislocated columns back into place.  
# This mends the rows, but we could alternatively just take care of this manually in the csv using a text editor.

# Shift the cells that were mislocated in their entirety.
df.iloc[3600, 3:11] = df.iloc[3601, 1:9]
# Join the cell that was broken in the middle.
df.iloc[3600, 2] = df.iloc[3600, 2] + '\n' + df.iloc[3601, 0]

df.iloc[16061, 3:11] = df.iloc[16062, 1:9]
df.iloc[16061, 2] = df.iloc[16061, 2] + '\n' + df.iloc[16062, 0]

df.drop(df.index[[3601,16062]], inplace = True)

In [10]:
# Check the mended rows.
df.iloc[[3600, 16060], :]

,tweet_id,created_at,full_text,geo,coordinates,place,retweet_count,favorite_count,possibly_sensitive,lang,user_id
3600,1364723003620270083,Wed Feb 24 23:44:54 +0000 2021,Good News for Europeans... even if your leftis...,NaN,NaN,NaN,1,1,False,en,46902592
16061,1366124610568216582,Sun Feb 28 20:34:23 +0000 2021,Important Open Letter and Analysis of the Covi...,NaN,NaN,NaN,0,1,False,en,46902592.0


In [11]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51170 entries, 0 to 51171
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            51170 non-null  object
 1   created_at          51170 non-null  object
 2   full_text           51170 non-null  object
 3   geo                 77 non-null     object
 4   coordinates         77 non-null     object
 5   place               1006 non-null   object
 6   retweet_count       51170 non-null  object
 7   favorite_count      51170 non-null  object
 8   possibly_sensitive  51170 non-null  object
 9   lang                51170 non-null  object
 10  user_id             51170 non-null  object
dtypes: object(11)
memory usage: 4.7+ MB


### Check for duplicates

In [12]:
df.duplicated().value_counts()

False    51157
True        13
dtype: int64

In [13]:
suspected_duplicates_df = df[df.duplicated(keep = False)]
len(suspected_duplicates_df)

26

In [14]:
suspected_duplicates_df.sort_values('tweet_id')

,tweet_id,created_at,full_text,geo,coordinates,place,retweet_count,favorite_count,possibly_sensitive,lang,user_id
51171,1364320252461535235,Tue Feb 23 21:04:30 +0000 2021,"@CGreenUK No, but maybe employers shouldn't be...",NaN,NaN,NaN,0,0,NOT FOUND,en,747351737443328000.0
9061,1364320252461535235,Tue Feb 23 21:04:30 +0000 2021,"@CGreenUK No, but maybe employers shouldn't be...",NaN,NaN,NaN,0,0,NOT FOUND,en,747351737443328000.0
9060,1364519307103916039,Wed Feb 24 10:15:29 +0000 2021,@NickBoles Treating the Covid19 vaccines diffe...,NaN,NaN,NaN,0,2,NOT FOUND,en,3049083785.0
51170,1364519307103916039,Wed Feb 24 10:15:29 +0000 2021,@NickBoles Treating the Covid19 vaccines diffe...,NaN,NaN,NaN,0,2,NOT FOUND,en,3049083785.0
9059,1364573246885163013,Wed Feb 24 13:49:49 +0000 2021,@BarryCPearson @HilaryWallace1 @stiffenking @l...,NaN,NaN,NaN,0,3,False,en,19769417.0
51169,1364573246885163013,Wed Feb 24 13:49:49 +0000 2021,@BarryCPearson @HilaryWallace1 @stiffenking @l...,NaN,NaN,NaN,0,3,False,en,19769417.0
51168,1364594670030962691,Wed Feb 24 15:14:56 +0000 2021,@TamronHallShow please don't put Anti-Vaxxers ...,NaN,NaN,NaN,0,0,NOT FOUND,en,120200107.0
9058,1364594670030962691,Wed Feb 24 15:14:56 +0000 2021,@TamronHallShow please don't put Anti-Vaxxers ...,NaN,NaN,NaN,0,0,NOT FOUND,en,120200107.0
9057,1364598020558491655,Wed Feb 24 15:28:15 +0000 2021,Remember when you saw Toby Young and Julia Har...,NaN,NaN,NaN,0,0,False,en,2548402543.0
51167,1364598020558491655,Wed Feb 24 15:28:15 +0000 2021,Remember when you saw Toby Young and Julia Har...,NaN,NaN,NaN,0,0,False,en,2548402543.0


In [15]:
# Those all look like legitimate duplicates. Let's drop them.
df.drop_duplicates(inplace=True)

In [16]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51157 entries, 0 to 51158
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            51157 non-null  object
 1   created_at          51157 non-null  object
 2   full_text           51157 non-null  object
 3   geo                 77 non-null     object
 4   coordinates         77 non-null     object
 5   place               1005 non-null   object
 6   retweet_count       51157 non-null  object
 7   favorite_count      51157 non-null  object
 8   possibly_sensitive  51157 non-null  object
 9   lang                51157 non-null  object
 10  user_id             51157 non-null  object
dtypes: object(11)
memory usage: 4.7+ MB


### Recast column datatypes

In [17]:
# Tidy up the column Dtypes
# 'tweet_id' is too big for int or long.

df['tweet_id']=df.tweet_id.astype('float64')
df['retweet_count']=df.retweet_count.astype('int64')
df['favorite_count']=df.favorite_count.astype('int64')
df['user_id']=df.user_id.astype('float64')

In [18]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51157 entries, 0 to 51158
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            51157 non-null  float64
 1   created_at          51157 non-null  object 
 2   full_text           51157 non-null  object 
 3   geo                 77 non-null     object 
 4   coordinates         77 non-null     object 
 5   place               1005 non-null   object 
 6   retweet_count       51157 non-null  int64  
 7   favorite_count      51157 non-null  int64  
 8   possibly_sensitive  51157 non-null  object 
 9   lang                51157 non-null  object 
 10  user_id             51157 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 4.7+ MB


### Adding datetime for 'created_at'

In [19]:
# Add a column for 'created_at' recast as datetime.
# I kept the original 'created_at' column out of a super-abundance of caution.

df['created_at'] = pd.to_datetime(df['created_at'])

In [20]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51157 entries, 0 to 51158
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            51157 non-null  float64            
 1   created_at          51157 non-null  datetime64[ns, UTC]
 2   full_text           51157 non-null  object             
 3   geo                 77 non-null     object             
 4   coordinates         77 non-null     object             
 5   place               1005 non-null   object             
 6   retweet_count       51157 non-null  int64              
 7   favorite_count      51157 non-null  int64              
 8   possibly_sensitive  51157 non-null  object             
 9   lang                51157 non-null  object             
 10  user_id             51157 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(2), object(6)
memory usage: 4.7+ MB


In [21]:
df.head()

,tweet_id,created_at,full_text,geo,coordinates,place,retweet_count,favorite_count,possibly_sensitive,lang,user_id
0,1.364223e+18,2021-02-23 14:38:16+00:00,Here’s what's in the COVID relief package:\n \...,NaN,NaN,NaN,9160,38093,NOT FOUND,en,2.950125e+07
1,1.364381e+18,2021-02-24 01:07:52+00:00,Will the National Endowment for the Arts be he...,NaN,NaN,NaN,6131,18560,NOT FOUND,en,1.201671e+18
2,1.364610e+18,2021-02-24 16:14:15+00:00,"This is both anecdotal and early, but many lon...",NaN,NaN,NaN,5941,63174,NOT FOUND,en,3.842872e+07
3,1.364727e+18,2021-02-24 23:59:58+00:00,A Link to Professor Chossudovsky’s Analysis of...,NaN,NaN,NaN,1,0,False,en,2.192010e+09
4,1.364727e+18,2021-02-24 23:59:58+00:00,Children warned over hugging grandparents even...,NaN,NaN,NaN,0,2,False,en,2.868190e+09


### Export the cleaned data

In [22]:
# Write it to csv.

df.to_csv(clean_csv, index = False)